<a href="https://colab.research.google.com/github/fjadidi2001/fake_news_detection/blob/main/DANSE_Mar26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Step-by-Step Workflow to Apply DANES Methodology on `facebook-fact-check.csv` Dataset**

---

## **📌 Step 1: Install Required Libraries**

---

---
## **📌 Step 2: Load and Explore the Dataset**
Start by loading and understanding your dataset.

In [5]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/facebook-fact-check.csv", encoding='latin1')

# Check initial shape and missing values
print("Initial shape:", df.shape)  # Should be (2282, 14)
print("\nMissing values:\n", df.isnull().sum())

Initial shape: (2282, 13)

Missing values:
 account_id           0
post_id              0
Category             0
Page                 0
Post URL             0
Date Published       0
Post Type            0
Rating               0
Debate            1984
share_count         70
reaction_count       2
comment_count        2
Context Post         0
dtype: int64


### 🔹 **Check for Missing Values**
---

---
## **📌 Step 3: Define Target Variable**
- If `Rating` column contains fact-checking labels, convert it into a binary/multi-class target variable.
---

---

## **📌 Step 4: Preprocess Text Data (Text Branch)**
Since DANES uses deep learning models, we need to **clean, tokenize, and embed** the text.

### 🔹 **Convert Text to Word Embeddings**
Use **Word2Vec, FastText, or GloVe** to obtain word embeddings.

---


---

## **📌 Step 5: Prepare Social Context Features (Social Branch)**
We normalize numerical social features (`share_count`, `reaction_count`, `comment_count`).

---

---
## **📌 Step 6: Train-Test Split**

---

---
## **📌 Step 7: Build the DANES Model (Deep Learning)**
 Create the **Text Branch (LSTM)** and **Social Branch (MLP/CNN)** and combine them.

---

---
## **📌 Step 8: Train the Model**

---

## **📌 Step 9: Evaluate the Model**

---

## **📌 Step 10: Save the Model for Future Use**

---

In [7]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [8]:
# Step 1: Installations (already done in Colab)

# Step 2: Load and Explore
import pandas as pd
df = pd.read_csv("/content/facebook-fact-check.csv", encoding='latin1')

# Step 3: Multi-class Target Variable
from sklearn.preprocessing import LabelEncoder

# Clean Rating column
df['Rating'] = df['Rating'].str.strip().str.lower()
print("Unique Ratings:", df['Rating'].unique())

# Encode labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['Rating'])
print("\nLabel mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

# Step 4: Preprocess Text with GloVe Embeddings
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

df['clean_text'] = df['Context Post'].apply(preprocess_text)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])
max_len = 150  # Adjusted based on text length analysis
X_text = pad_sequences(sequences, maxlen=max_len, padding='post')

# Load GloVe Embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

embedding_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Create embedding matrix
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Step 5: Social Context Features
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Select and impute social features
social_features = df[['share_count', 'reaction_count', 'comment_count']]
imputer = SimpleImputer(strategy='median')
social_imputed = imputer.fit_transform(social_features)

# Scale features
scaler = StandardScaler()
X_social = scaler.fit_transform(social_imputed)

# Step 6: Train-Test Split
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Convert labels to categorical
y = to_categorical(df['label'])

X_text_train, X_text_test, X_social_train, X_social_test, y_train, y_test = train_test_split(
    X_text, X_social, y, test_size=0.2, stratify=y, random_state=42
)

# Step 7: Build DANES Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Dropout

# Text Branch
text_input = Input(shape=(max_len,))
x = Embedding(vocab_size, embedding_dim,
             weights=[embedding_matrix],
             input_length=max_len,
             trainable=False)(text_input)
x = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(x)

# Social Branch
social_input = Input(shape=(3,))
y = Dense(64, activation='relu')(social_input)
y = Dropout(0.3)(y)

# Combined Model
combined = Concatenate()([x, y])
z = Dense(128, activation='relu')(combined)
z = Dropout(0.4)(z)
output = Dense(len(le.classes_), activation='softmax')(z)

model = Model(inputs=[text_input, social_input], outputs=output)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Step 8: Train the Model
history = model.fit(
    [X_text_train, X_social_train], y_train,
    epochs=15,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

# Step 9: Evaluate
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# Plot training history
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Training History')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Evaluate model
loss, accuracy = model.evaluate([X_text_test, X_social_test], y_test, verbose=0)
print(f'Test Accuracy: {accuracy:.4f}')

# Generate predictions
y_pred = model.predict([X_text_test, X_social_test])
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=le.classes_))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_classes, y_pred_classes))

# Step 10: Save Model
model.save('danes_multiclass.h5')

Unique Ratings: ['no factual content' 'mostly true' 'mixture of true and false'
 'mostly false']

Label mapping: {'mixture of true and false': np.int64(0), 'mostly false': np.int64(1), 'mostly true': np.int64(2), 'no factual content': np.int64(3)}


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


--2025-03-26 20:28:59--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-03-26 20:28:59--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-03-26 20:28:59--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 18s 426ms/step - accuracy: 0.6511 - loss: 1.2325 - val_accuracy: 0.7370 - val_loss: 0.8316
Epoch 2/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 477ms/step - accuracy: 0.7323 - loss: 0.8576 - val_accuracy: 0.7452 - val_loss: 0.8108
Epoch 3/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 20s 468ms/step - accuracy: 0.7494 - loss: 0.8093 - val_accuracy: 0.7397 - val_loss: 0.8126
Epoch 4/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 20s 424ms/step - accuracy: 0.7420 - loss: 0.8289 - val_accuracy: 0.7370 - val_loss: 0.8097
Epoch 5/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 467ms/step - accuracy: 0.7413 - loss: 0.8107 - val_accuracy: 0.7342 - val_loss: 0.8091
Epoch 6/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 479ms/step - accuracy: 0.7396 - loss: 0.8338 - val_accuracy: 0.7397 - val_loss: 0.7901
Epoch 7/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 18s 387ms/step - accuracy: 0.7502 - loss: 0.7940 - val_accuracy: 0.7342 - val_loss: 0.7979
Epoch 8/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 478ms/step - accuracy: 0.7419 - loss: 0.8352 - val_accu

Test Accuracy: 0.7396
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Classification Report:
                           precision    recall  f1-score   support

mixture of true and false       0.00      0.00      0.00        49
             mostly false       0.00      0.00      0.00        21
              mostly true       0.75      0.98      0.85       334
       no factual content       0.61      0.21      0.31        53

                 accuracy                           0.74       457
                macro avg       0.34      0.30      0.29       457
             weighted avg       0.62      0.74      0.66       457


Confusion Matrix:
[[  0   0  48   1]
 [  0   0  21   0]
 [  1   0 327   6]
 [  0   0  42  11]]


In [15]:
# Step 1: Installations
!python -m spacy download en_core_web_sm

# Step 2: Load and Explore Dataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

df = pd.read_csv("/content/facebook-fact-check.csv", encoding='latin-1')

# Step 3: Multi-class Target Variable
rating_mapping = {
    'mostly true': 0,
    'mixture of true and false': 1,
    'no factual content': 2,
    'mostly false': 3
}
df['label'] = df['Rating'].str.strip().str.lower().map(rating_mapping)

# Step 4: Advanced Text Preprocessing
from transformers import AutoTokenizer

# from tokenization import Tokenization  # From your source code

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Update the code to use the new tokenizer
tkn = tokenizer # or how you want to use it
corpus = [tkn.tokenize(str(text)) #adjust this line according to the new tokenizer
          for text in df['Context Post']]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!pip show word_embeddings_sdk # Review the package information, including the location


Name: word-embeddings-sdk
Version: 0.1.5
Summary: Python sdk to interface with the WordEmbeddings API
Home-page: https://github.com/Width-ai/embeddings-sdk
Author: Patrick Hennis
Author-email: patrick@width.ai
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: pydantic, requests
Required-by: 


In [2]:
!pip install word-embeddings-sdk  # Install the wordembeddings package
# Step 5: Word Embeddings Implementation
from word_embeddings_sdk import WordEmbeddings  # From your source code

we = WordEmbeddings([corpus])  # Wrap corpus in list to match source structure
docs = we.prepareDocuments()

# Get all embeddings
embeddings = {
    'w2v_cbow': we.word2vecEmbedding(sg=0),
    'w2v_sg': we.word2vecEmbedding(sg=1),
    'ft_cbow': we.word2FastTextEmbeddings(sg=0),
    'ft_sg': we.word2FastTextEmbeddings(sg=1),
    'glove': we.word2GloVeEmbedding(),
    'mittens': we.word2MittensEmbedding()
}

docs = we.preprareDocuments()

# Get all embeddings
embeddings = {
    'w2v_cbow': we.word2vecEmbedding(sg=0),
    'w2v_sg': we.word2vecEmbedding(sg=1),
    'ft_cbow': we.word2FastTextEmbeddings(sg=0),
    'ft_sg': we.word2FastTextEmbeddings(sg=1),
    'glove': we.word2GloVeEmbedding(),
    'mittens': we.word2MittensEmbedding()
}

ImportError: cannot import name 'WordEmbeddings' from 'word_embeddings_sdk' (/usr/local/lib/python3.11/dist-packages/word_embeddings_sdk/__init__.py)

In [14]:



# Step 6: Social Feature Processing
social_features = df[['share_count', 'reaction_count', 'comment_count']]
social_features = social_features.fillna(social_features.median())

# Step 7: Model Architecture (Modified from source code)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Conv1D, Flatten, Bidirectional

def create_danes_model(embedding_matrix, vocab_size, max_len, social_dim, num_classes):
    # Text Branch
    text_input = Input(shape=(max_len,))
    x = Embedding(vocab_size, embedding_matrix.shape[1],
                 weights=[embedding_matrix],
                 trainable=False)(text_input)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Conv1D(64, 3, activation='relu')(x)
    x = Flatten()(x)

    # Social Branch
    social_input = Input(shape=(social_dim,))
    y = Dense(64, activation='relu')(social_input)

    # Combined
    combined = Concatenate()([x, y])
    z = Dense(128, activation='relu')(combined)
    output = Dense(num_classes, activation='softmax')(z)

    return Model(inputs=[text_input, social_input], outputs=output)

# Step 8: Training Framework
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping

def train_and_evaluate(model, X_text_train, X_social_train, y_train,
                       X_text_val, X_social_val, y_val,
                       X_text_test, X_social_test, y_test):
    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

    es = EarlyStopping(patience=5, restore_best_weights=True)

    history = model.fit(
        [X_text_train, X_social_train], y_train,
        validation_data=([X_text_val, X_social_val], y_val),
        epochs=50,
        batch_size=64,
        callbacks=[es],
        verbose=1
    )

    # Evaluation
    y_pred = model.predict([X_text_test, X_social_test])
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    print(classification_report(y_test_classes, y_pred_classes))
    return history

# Step 9: Cross-validated Training
from sklearn.preprocessing import StandardScaler

results = []
for emb_name, emb_matrix in embeddings.items():
    print(f"\n=== Training with {emb_name} embeddings ===")

    # Prepare text data
    X_text = pad_sequences(docs, maxlen=we.max_size, padding='post')

    # Prepare social features
    scaler = StandardScaler()
    X_social = scaler.fit_transform(social_features)

    # Convert labels
    y = to_categorical(df['label'])

    # Train-test split
    X_text_train, X_text_test, X_social_train, X_social_test, y_train, y_test = \
        train_test_split(X_text, X_social, y, test_size=0.2, stratify=y)

    # Create and train model
    model = create_danes_model(
        emb_matrix,
        vocab_size=emb_matrix.shape[0],
        max_len=we.max_size,
        social_dim=X_social.shape[1],
        num_classes=y.shape[1]
    )

    history = train_and_evaluate(
        model,
        X_text_train, X_social_train, y_train,
        X_text_test, X_social_test, y_test,
        X_text_test, X_social_test, y_test
    )

    # Store results
    results.append({
        'embedding': emb_name,
        'model': model,
        'history': history,
        'vocab_size': emb_matrix.shape[0],
        'max_len': we.max_size
    })

# Step 10: Model Saving and Visualization
import matplotlib.pyplot as plt

for result in results:
    # Save model
    result['model'].save(f"danes_model_{result['embedding']}.h5")

    # Plot training history
    plt.figure()
    plt.plot(result['history'].history['accuracy'], label='Train Accuracy')
    plt.plot(result['history'].history['val_accuracy'], label='Validation Accuracy')
    plt.title(f"Training History - {result['embedding']}")
    plt.legend()
    plt.show()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
ERROR: Could not find a version that satisfies the requirement wordembeddings (from versions: none)
ERROR: No matching distribution found for wordembeddings


ModuleNotFoundError: No module named 'wordembeddings'

In [10]:
!pip install tokenization

In [11]:
!pip install tokenizers